<a href="https://colab.research.google.com/github/Kenny625819/Applied-Data-Science/blob/main/ESJ_final_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# ESJ COMPLETE PAPER VERSION (SUBMISSION FINAL) — UPDATED
# ✅ Full vs Preop-only (same complete-case cohort)
# ✅ CV OOF + Isotonic (no leakage)
# ✅ Temporal validation (TRAIN OOF isotonic -> TEST)
# ✅ ROC (square) + DCA (legend not cut)
# ✅ Calibration plots (Full & Preop; 3/6/12M)
# ✅ SHAP Top7 + SHAP heatmaps (Full & Preop)
#
# ===========================
# USER REQUESTED UPDATES
# ① DCA figsize = (8, 6) inch (font sizes unchanged)
# ② SHAP Top7 fonts: xlabel 20, xticks 20, yticks 20
# ③ SHAP Heatmap: remove title
# ④ SHAP Heatmap fonts: xticks 20, yticks 20, colorbar 18
# ⑤ Calibration: keep the current approach (quantile bins) as reference
# ⑥ Calibration fonts: xlabel 20, ylabel 20
# ⑦ Calibration figsize = (6, 6) inch
# ============================================================

!pip -q install lightgbm shap scipy openpyxl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score, brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb
import shap
from scipy import stats

# -----------------------------
# SETTINGS
# -----------------------------
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_PATH = Path("/content/patient All2013.xlsx")
if not DATA_PATH.exists():
    DATA_PATH = Path("/mnt/data/patient All2013.xlsx")
if not DATA_PATH.exists():
    raise FileNotFoundError("patient All2013.xlsx not found in /content or /mnt/data")

SHEET_NAME = "Sheet1"
OUT_DIR = Path("/content/ESJ_outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)

N_SPLITS = 5
N_BOOT = 2000

TRAIN_YEARS = (2013, 2016)
TEST_YEARS  = (2017, 2021)

N_PRIMARY_TESTS = 6
ALPHA_PRIMARY_BONF = 0.05 / N_PRIMARY_TESTS

DCA_MIN, DCA_MAX, DCA_NPTS = 0.05, 0.50, 19

ROC_FS = 20
LEG_FS = 18  # legend fontsize (ROC/DCA)

SHAP_XMAX = 2.5

# ✅ Requested fonts
TOP7_TICK_FS   = 20
TOP7_XLABEL_FS = 20

CAL_LABEL_FS = 20
CAL_TICK_FS  = 20  # not requested explicitly, but keeps consistency/readability

HM_X_FS   = 20
HM_Y_FS   = 20
HM_CBAR_FS = 18

plt.rcParams.update({"font.family": "DejaVu Sans", "axes.unicode_minus": False})

# -----------------------------
# COLUMNS
# -----------------------------
DATE_COL = "ope date"
Y_COLS = {"3M":"3Month Survival", "6M":"6Month Survival", "12M":"12Month Survival"}
TOK_COL = "Revised Tokuhashi score"
KAT_COL = "New Katagiri score"

PREOP_FEATURES = [
    "Age", "Sex", "BMI",
    "Malignancy (Katagiri Score)",
    "Visceral Metastasis",
    "Number of Spinal Metastases",
    "ECOGPS",
    "Frankel_bin",
    "Barthel Index",
    "Serum Albumin",
    "CRP",
]
INTRAOP_FEATURES = ["Operation Time", "Intraoperative Blood Loss"]
FULL_FEATURES = PREOP_FEATURES + INTRAOP_FEATURES
ECOG_COL = "ECOGPS"

# -----------------------------
# ESJ-short labels (figures only)
# -----------------------------
DISPLAY_NAME_MAP = {
    "Age": "Age",
    "Sex": "Sex",
    "BMI": "BMI",
    "Malignancy (Katagiri Score)": "Malignancy",
    "Visceral Metastasis": "Visceral mets",
    "Number of Spinal Metastases": "Spinal mets",
    "ECOGPS": "ECOG PS",
    "Frankel_bin": "Frankel grade",
    "Barthel Index": "Barthel index",
    "Serum Albumin": "Albumin",
    "CRP": "CRP",
    "Operation Time": "Operative time",
    "Intraoperative Blood Loss": "Blood loss",
}
def apply_display_names(cols):
    return [DISPLAY_NAME_MAP.get(c, c) for c in cols]

# -----------------------------
# Helpers
# -----------------------------
def minmax_01(x):
    x = np.asarray(x, dtype=float)
    mn, mx = np.min(x), np.max(x)
    if mx - mn < 1e-12:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def make_frankel_bin(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.upper().str.strip()
    return s.map({"A":0,"B":0,"C":0,"D":1,"E":1}).astype(float)

def build_X(df, feature_cols):
    X = df[feature_cols].copy()
    if ECOG_COL in X.columns:
        X[ECOG_COL] = pd.to_numeric(X[ECOG_COL], errors="coerce").astype(float)
    return X

def align_train_test(Xtr, Xte):
    Xtr, Xte = Xtr.align(Xte, join="outer", axis=1, fill_value=0)
    return Xtr, Xte

def bootstrap_auc_ci(y_true, y_score, n_boot=2000, seed=42):
    rng = np.random.default_rng(seed)
    y_true = np.asarray(y_true)
    y_score = np.asarray(y_score)
    n = len(y_true)
    aucs = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        if len(np.unique(y_true[idx])) < 2:
            continue
        aucs.append(roc_auc_score(y_true[idx], y_score[idx]))
    auc = roc_auc_score(y_true, y_score)
    if len(aucs) == 0:
        return float(auc), float("nan"), float("nan")
    lo, hi = np.percentile(np.array(aucs), [2.5, 97.5])
    return float(auc), float(lo), float(hi)

def calibration_slope_intercept(y_true, p_cal):
    y_true = np.asarray(y_true).astype(int)
    p = np.clip(np.asarray(p_cal, dtype=float), 1e-6, 1-1e-6)
    log_odds = np.log(p/(1-p)).reshape(-1,1)
    lr = LogisticRegression(penalty=None, solver="lbfgs", max_iter=1000)
    lr.fit(log_odds, y_true)
    return float(lr.coef_[0][0]), float(lr.intercept_[0])

def best_threshold_youden(y_true, y_score):
    fpr, tpr, thr = roc_curve(y_true, y_score)
    j = tpr - fpr
    k = int(np.argmax(j))
    return float(thr[k])

def metrics_at_threshold(y_true, y_prob, thr):
    y_pred = (np.asarray(y_prob) >= float(thr)).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sens = tp / (tp + fn) if (tp + fn) else np.nan
    spec = tn / (tn + fp) if (tn + fp) else np.nan
    f1 = f1_score(y_true, y_pred)
    return float(sens), float(spec), float(f1)

# -----------------------------
# CV OOF + Isotonic (NO train AUC)
# -----------------------------
def run_lgb_cv_oof(X, y, lgb_params, n_splits=5, seed=42, n_boot=2000):
    y = np.asarray(y).astype(int)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    p_oof = np.zeros(len(y), dtype=float)
    for tr, te in skf.split(X, y):
        model = lgb.LGBMClassifier(**lgb_params)
        model.fit(X.iloc[tr], y[tr])
        p_oof[te] = model.predict_proba(X.iloc[te])[:,1]

    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(p_oof, y)
    p_cal = iso.transform(p_oof)

    auc, lo, hi = bootstrap_auc_ci(y, p_cal, n_boot=n_boot, seed=seed)
    thr = best_threshold_youden(y, p_cal)
    sens, spec, f1 = metrics_at_threshold(y, p_cal, thr)
    brier = float(brier_score_loss(y, p_cal))
    cal_slope, cal_intercept = calibration_slope_intercept(y, p_cal)

    return dict(
        p_oof=p_oof, p_cal=p_cal, iso=iso,
        auc=auc, auc_lo=lo, auc_hi=hi,
        sens=sens, spec=spec, f1=f1, brier=brier,
        thr=thr,
        cal_slope=cal_slope, cal_intercept=cal_intercept
    )

# -----------------------------
# Temporal validation (TRAIN OOF isotonic -> TEST)
# -----------------------------
def run_temporal_validation(df_all, y_col, feature_cols, lgb_params, train_years, test_years, seed=42):
    df_all = df_all.copy()
    df_all["year"] = pd.to_datetime(df_all[DATE_COL]).dt.year
    tr0, tr1 = train_years
    te0, te1 = test_years

    dtr = df_all[(df_all["year"] >= tr0) & (df_all["year"] <= tr1)].copy()
    dte = df_all[(df_all["year"] >= te0) & (df_all["year"] <= te1)].copy()

    Xtr = build_X(dtr, feature_cols)
    Xte = build_X(dte, feature_cols)
    Xtr, Xte = align_train_test(Xtr, Xte)

    ytr = dtr[y_col].astype(int).values
    yte = dte[y_col].astype(int).values

    cv_tr = run_lgb_cv_oof(Xtr, ytr, lgb_params, n_splits=N_SPLITS, seed=seed, n_boot=200)
    iso = cv_tr["iso"]
    thr_train = cv_tr["thr"]

    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(Xtr, ytr)
    p_raw = model.predict_proba(Xte)[:,1]
    p_cal = iso.transform(p_raw)

    auc_raw = float(roc_auc_score(yte, p_raw)) if len(np.unique(yte)) == 2 else float("nan")
    auc_cal = float(roc_auc_score(yte, p_cal)) if len(np.unique(yte)) == 2 else float("nan")
    sens, spec, f1 = metrics_at_threshold(yte, p_cal, thr_train)
    brier = float(brier_score_loss(yte, p_cal))
    slope, intercept = calibration_slope_intercept(yte, p_cal)

    return dict(
        train_years=f"{train_years[0]}–{train_years[1]}",
        test_years=f"{test_years[0]}–{test_years[1]}",
        n_test=int(len(yte)),
        deaths_test=int((yte==0).sum()),
        auc_raw=auc_raw, auc_cal=auc_cal,
        sens_cal=sens, spec_cal=spec, f1_cal=f1,
        brier_cal=brier,
        cal_slope=slope, cal_intercept=intercept,
        p_test_cal=p_cal
    )

# -----------------------------
# DeLong
# -----------------------------
def _compute_midrank(x):
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i+j-1) + 1
        i = j
    T2 = np.empty(N, dtype=float)
    T2[J] = T
    return T2

def _fast_delong(preds_sorted_transposed, label_1_count):
    m = label_1_count
    n = preds_sorted_transposed.shape[1] - m
    pos = preds_sorted_transposed[:, :m]
    neg = preds_sorted_transposed[:, m:]
    k = preds_sorted_transposed.shape[0]
    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m+n], dtype=float)
    for r in range(k):
        tx[r,:] = _compute_midrank(pos[r,:])
        ty[r,:] = _compute_midrank(neg[r,:])
        tz[r,:] = _compute_midrank(preds_sorted_transposed[r,:])
    aucs = (tz[:, :m].sum(axis=1) - m*(m+1)/2) / (m*n)
    v01 = (tz[:, :m] - tx) / n
    v10 = 1.0 - (tz[:, m:] - ty) / m
    sx = np.atleast_2d(np.cov(v01))
    sy = np.atleast_2d(np.cov(v10))
    s = sx/m + sy/n
    return aucs, s

def delong_pvalue(y_true, s1, s2):
    y_true = np.asarray(y_true).astype(int)
    order = np.argsort(-y_true)
    y_sorted = y_true[order]
    preds = np.vstack([s1, s2])[:, order]
    m = int(y_sorted.sum())
    aucs, s = _fast_delong(preds, m)
    diff = aucs[0] - aucs[1]
    var = s[0,0] + s[1,1] - 2*s[0,1]
    z = diff / np.sqrt(var + 1e-12)
    p = 2*(1 - stats.norm.cdf(abs(z)))
    return float(p), float(aucs[0]), float(aucs[1])

def bonferroni_adjust(p, m):
    return float(min(p*m, 1.0))

# -----------------------------
# PLOTS (ROC square, DCA resized)
# -----------------------------
def plot_roc_three_square(y, p_ai, s_tok_surv, s_kat_surv, save_path):
    auc_ai  = roc_auc_score(y, p_ai)
    auc_tok = roc_auc_score(y, s_tok_surv)
    auc_kat = roc_auc_score(y, s_kat_surv)

    fig, ax = plt.subplots(figsize=(6,6))
    ax.plot([0,1],[0,1],"--",color="gray")

    fpr,tpr,_ = roc_curve(y,p_ai)
    ax.plot(fpr,tpr,"-",color="black",linewidth=2.5,label=f"AI (LightGBM), AUC={auc_ai:.3f}")

    fpr2,tpr2,_ = roc_curve(y,s_tok_surv)
    ax.plot(fpr2,tpr2,"--",color="black",linewidth=2,label=f"Revised Tokuhashi, AUC={auc_tok:.3f}")

    fpr3,tpr3,_ = roc_curve(y,s_kat_surv)
    ax.plot(fpr3,tpr3,":",color="black",linewidth=2,label=f"New Katagiri, AUC={auc_kat:.3f}")

    ax.set_xlabel("1 – Specificity",fontsize=ROC_FS)
    ax.set_ylabel("Sensitivity",fontsize=ROC_FS)
    ax.tick_params(labelsize=ROC_FS)
    ax.set_aspect('equal', adjustable='box')

    leg = ax.legend(fontsize=LEG_FS, loc="center left", bbox_to_anchor=(1.02,0.5), frameon=True)
    leg.get_frame().set_edgecolor("black")
    leg.get_frame().set_linewidth(1.2)

    fig.subplots_adjust(right=0.75)
    fig.savefig(save_path, dpi=600, bbox_inches="tight", pad_inches=0.2, bbox_extra_artists=[leg])
    plt.close(fig)

def decision_curve_net_benefit(y_event, p_event, thresholds):
    y_event = np.asarray(y_event).astype(int)
    p_event = np.asarray(p_event).astype(float)
    n = len(y_event)
    nb = []
    for pt in thresholds:
        pred = (p_event >= pt).astype(int)
        tp = np.sum((pred==1) & (y_event==1))
        fp = np.sum((pred==1) & (y_event==0))
        w = pt/(1-pt)
        nb.append(tp/n - (fp/n)*w)
    return np.array(nb)

def decision_curve_baselines(y_event, thresholds):
    prev = np.mean(np.asarray(y_event).astype(int) == 1)
    nb_none = np.zeros_like(thresholds, dtype=float)
    nb_all  = prev - (1-prev)*(thresholds/(1-thresholds))
    return nb_none, nb_all

def plot_dca_multi_wide(y_event, curves_dict, save_path):
    thresholds = np.linspace(DCA_MIN, DCA_MAX, DCA_NPTS)
    nb_none, nb_all = decision_curve_baselines(y_event, thresholds)

    # ✅ ① figsize=(8,6), fonts unchanged
    fig, ax = plt.subplots(figsize=(8,6))
    ax.plot(thresholds, nb_none, "--", linewidth=2, label="Treat none", color="black")
    ax.plot(thresholds, nb_all,  "-", linewidth=2, label="Treat all",  color="black")

    for label, p_event in curves_dict.items():
        nb = decision_curve_net_benefit(y_event, p_event, thresholds)
        ax.plot(thresholds, nb, "-", linewidth=2, label=label)

    ax.set_xlabel("Threshold probability", fontsize=ROC_FS)
    ax.set_ylabel("Net benefit", fontsize=ROC_FS)
    ax.tick_params(labelsize=ROC_FS)

    ymin, ymax = ax.get_ylim()
    start = np.floor(ymin*10)/10
    end   = np.ceil(ymax*10)/10
    ax.set_yticks(np.arange(start, end + 0.0001, 0.1))

    leg = ax.legend(fontsize=LEG_FS, loc="center left", bbox_to_anchor=(1.02,0.5), frameon=True)
    leg.get_frame().set_edgecolor("black")
    leg.get_frame().set_linewidth(1.2)

    fig.subplots_adjust(right=0.75)
    fig.savefig(save_path, dpi=600, bbox_inches="tight", pad_inches=0.2, bbox_extra_artists=[leg])
    plt.close(fig)

# -----------------------------
# Calibration plots (Full & Preop) — UPDATED (⑥⑦)
# -----------------------------
def plot_calibration(y, p, save_path, bins=10):
    """
    Quantile-binned calibration (qcut), following your reference style.
    """
    dfc = pd.DataFrame({"y": np.asarray(y).astype(int), "p": np.asarray(p).astype(float)})
    dfc["bin"] = pd.qcut(dfc["p"], q=bins, duplicates="drop")
    g = dfc.groupby("bin").agg(obs=("y","mean"), pred=("p","mean"))

    # ✅ ⑦ figsize=(6,6)
    fig, ax = plt.subplots(figsize=(6,6))
    ax.plot([0,1],[0,1],"--",color="gray")
    ax.plot(g["pred"], g["obs"], "o-", color="black", linewidth=2)

    # ✅ ⑥ label fonts 20/20
    ax.set_xlabel("Predicted survival probability", fontsize=CAL_LABEL_FS)
    ax.set_ylabel("Observed survival probability", fontsize=CAL_LABEL_FS)
    ax.tick_params(labelsize=CAL_TICK_FS)

    fig.tight_layout()
    fig.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.close(fig)

# -----------------------------
# SHAP Top7 — UPDATED (②)
# -----------------------------
def plot_shap_top7_boxed(model, X, save_path, x_max=2.5):
    X_disp = X.copy()
    X_disp.columns = apply_display_names(X_disp.columns)

    explainer = shap.TreeExplainer(model)
    sv = explainer.shap_values(X)
    sv = sv[1] if isinstance(sv, list) and len(sv)==2 else sv

    shap.summary_plot(sv, X_disp, plot_type="bar", max_display=7, show=False, color="dimgray")

    ax = plt.gca()
    plt.xlim(0, float(x_max))

    # ✅ ② tick and labels all 20
    plt.xticks(fontsize=TOP7_TICK_FS)
    plt.yticks(fontsize=TOP7_TICK_FS)
    plt.xlabel("mean(|SHAP value|)", fontsize=TOP7_XLABEL_FS)

    for side in ["top","right","left","bottom"]:
        ax.spines[side].set_visible(True)
        ax.spines[side].set_linewidth(1.2)
        ax.spines[side].set_color("black")
    ax.tick_params(width=1.2, colors="black")

    plt.tight_layout()
    plt.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.close()

# -----------------------------
# SHAP Heatmap (3 cols) — UPDATED (③④)
# -----------------------------
def shap_heatmap_3cols(models_by_tp, X_by_tp, save_path):
    cols = ["3M","6M","12M"]
    series_by_tp = {}
    for tp in cols:
        explainer = shap.TreeExplainer(models_by_tp[tp])
        sv = explainer.shap_values(X_by_tp[tp])
        sv = sv[1] if isinstance(sv, list) and len(sv)==2 else sv
        series_by_tp[tp] = pd.Series(np.abs(sv).mean(axis=0), index=X_by_tp[tp].columns)

    df_hm = pd.DataFrame(series_by_tp)
    df_hm.index = apply_display_names(df_hm.index)
    df_hm = df_hm.sort_values(by="3M", ascending=False)

    plt.figure(figsize=(10,12))
    plt.imshow(df_hm.values, cmap="Greys", aspect="auto")

    # ✅ ④ colorbar 18 (ticks + label)
    cbar = plt.colorbar(label="mean(|SHAP|)", fraction=0.035, pad=0.04)
    cbar.ax.tick_params(labelsize=HM_CBAR_FS)
    cbar.set_label("mean(|SHAP|)", fontsize=HM_CBAR_FS)

    # ✅ ④ X/Y label fonts
    plt.xticks(np.arange(3), cols, fontsize=HM_X_FS)
    plt.yticks(np.arange(len(df_hm.index)), df_hm.index, fontsize=HM_Y_FS)

    # ✅ ③ remove title (no plt.title)
    plt.tight_layout()
    plt.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.close()

# ============================================================
# LOAD & RUN
# ============================================================
df0 = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)

required = [DATE_COL, TOK_COL, KAT_COL, "Frankel Grade"] + list(Y_COLS.values()) + [c for c in FULL_FEATURES if c!="Frankel_bin"]
missing = [c for c in required if c not in df0.columns]
if missing:
    raise KeyError("Missing required columns:\n- " + "\n- ".join(missing))

df0["Frankel_bin"] = make_frankel_bin(df0["Frankel Grade"])

need = [DATE_COL, TOK_COL, KAT_COL] + list(Y_COLS.values()) + FULL_FEATURES
dff = df0.dropna(subset=need).copy()

X_full  = build_X(dff, FULL_FEATURES)
X_preop = build_X(dff, PREOP_FEATURES)
X_full, X_preop = align_train_test(X_full, X_preop)

tok_score_surv = dff[TOK_COL].astype(float).values
kat_score_surv = (-dff[KAT_COL].astype(float).values)
tok_risk01 = 1.0 - minmax_01(tok_score_surv)
kat_risk01 = minmax_01(dff[KAT_COL].astype(float).values)

LGB_PARAMS = dict(
    learning_rate=0.05,
    num_leaves=31,
    n_estimators=500,
    reg_lambda=1.0,
    class_weight="balanced",
    random_state=RANDOM_SEED,
)

rows_cv, rows_tv, rows_del = [], [], []
models_full, models_pre = {}, {}
X_full_tp, X_pre_tp = {}, {}

for tp, ycol in Y_COLS.items():
    y = dff[ycol].astype(int).values

    # CV (Full & Preop)
    res_full = run_lgb_cv_oof(X_full,  y, LGB_PARAMS, n_splits=N_SPLITS, seed=RANDOM_SEED, n_boot=N_BOOT)
    res_pre  = run_lgb_cv_oof(X_preop, y, LGB_PARAMS, n_splits=N_SPLITS, seed=RANDOM_SEED, n_boot=N_BOOT)

    # Score AUC + CI (for tables)
    tok_auc, tok_lo, tok_hi = bootstrap_auc_ci(y, tok_score_surv, n_boot=N_BOOT, seed=RANDOM_SEED)
    kat_auc, kat_lo, kat_hi = bootstrap_auc_ci(y, kat_score_surv, n_boot=N_BOOT, seed=RANDOM_SEED)

    # ROC (square) Full & Preop
    plot_roc_three_square(y, res_full["p_cal"], tok_score_surv, kat_score_surv, OUT_DIR / f"ROC_{tp}_Full.png")
    plot_roc_three_square(y, res_pre["p_cal"],  tok_score_surv, kat_score_surv, OUT_DIR / f"ROC_{tp}_PreopOnly.png")

    # DCA (event=death)
    y_event = (y == 0).astype(int)
    curves_full = {
        "AI model (Full)": 1.0 - res_full["p_cal"],
        "Revised Tokuhashi (scaled)": tok_risk01,
        "New Katagiri (scaled)": kat_risk01,
    }
    curves_pre = {
        "AI model (Preop-only)": 1.0 - res_pre["p_cal"],
        "Revised Tokuhashi (scaled)": tok_risk01,
        "New Katagiri (scaled)": kat_risk01,
    }
    plot_dca_multi_wide(y_event, curves_full, OUT_DIR / f"DCA_{tp}_Full.png")
    plot_dca_multi_wide(y_event, curves_pre,  OUT_DIR / f"DCA_{tp}_PreopOnly.png")

    # Calibration (Full & Preop)
    plot_calibration(y, res_full["p_cal"], OUT_DIR / f"Calibration_{tp}_Full.png")
    plot_calibration(y, res_pre["p_cal"],  OUT_DIR / f"Calibration_{tp}_PreopOnly.png")

    # DeLong (primary + supportive)
    p_ft, _, _ = delong_pvalue(y, res_full["p_cal"], tok_score_surv)
    p_fk, _, _ = delong_pvalue(y, res_full["p_cal"], kat_score_surv)
    p_fp, _, _ = delong_pvalue(y, res_full["p_cal"], res_pre["p_cal"])  # supportive

    rows_del.append({
        "Timepoint": tp,
        "p_Full_vs_Tokuhashi": p_ft,
        "p_Full_vs_Katagiri": p_fk,
        "p_Full_vs_Preop_supportive": p_fp,
        "p_Full_vs_Tokuhashi_BonferroniAdj": bonferroni_adjust(p_ft, N_PRIMARY_TESTS),
        "p_Full_vs_Katagiri_BonferroniAdj": bonferroni_adjust(p_fk, N_PRIMARY_TESTS),
        "Primary_alpha_Bonferroni": ALPHA_PRIMARY_BONF
    })

    # CV summary (NO Train AUC columns)
    rows_cv.append({
        "Timepoint": tp,
        "N": int(len(y)),
        "Deaths_by_timepoint": int((y==0).sum()),

        "Full_AUC": res_full["auc"], "Full_AUC_Lo": res_full["auc_lo"], "Full_AUC_Hi": res_full["auc_hi"],
        "Full_Sens": res_full["sens"], "Full_Spec": res_full["spec"], "Full_F1": res_full["f1"],
        "Full_Brier": res_full["brier"],
        "Full_CalSlope": res_full["cal_slope"], "Full_CalIntercept": res_full["cal_intercept"],

        "Preop_AUC": res_pre["auc"], "Preop_AUC_Lo": res_pre["auc_lo"], "Preop_AUC_Hi": res_pre["auc_hi"],
        "Preop_Sens": res_pre["sens"], "Preop_Spec": res_pre["spec"], "Preop_F1": res_pre["f1"],
        "Preop_Brier": res_pre["brier"],
        "Preop_CalSlope": res_pre["cal_slope"], "Preop_CalIntercept": res_pre["cal_intercept"],

        "Tokuhashi_AUC": tok_auc, "Tokuhashi_AUC_Lo": tok_lo, "Tokuhashi_AUC_Hi": tok_hi,
        "Katagiri_AUC": kat_auc, "Katagiri_AUC_Lo": kat_lo, "Katagiri_AUC_Hi": kat_hi,
    })

    # Temporal validation Full & Preop
    tv_full = run_temporal_validation(dff, ycol, FULL_FEATURES,  LGB_PARAMS, TRAIN_YEARS, TEST_YEARS, seed=RANDOM_SEED)
    tv_pre  = run_temporal_validation(dff, ycol, PREOP_FEATURES, LGB_PARAMS, TRAIN_YEARS, TEST_YEARS, seed=RANDOM_SEED)

    rows_tv.append({
        "Timepoint": tp,
        "TrainYears": tv_full["train_years"],
        "TestYears": tv_full["test_years"],
        "TestN": tv_full["n_test"],
        "TestDeaths": tv_full["deaths_test"],

        "Temporal_Full_AUC_raw": tv_full["auc_raw"],
        "Temporal_Full_AUC_cal": tv_full["auc_cal"],
        "Temporal_Full_Sens_cal": tv_full["sens_cal"],
        "Temporal_Full_Spec_cal": tv_full["spec_cal"],
        "Temporal_Full_F1_cal": tv_full["f1_cal"],
        "Temporal_Full_Brier_cal": tv_full["brier_cal"],
        "Temporal_Full_CalSlope": tv_full["cal_slope"],
        "Temporal_Full_CalIntercept": tv_full["cal_intercept"],

        "Temporal_Preop_AUC_raw": tv_pre["auc_raw"],
        "Temporal_Preop_AUC_cal": tv_pre["auc_cal"],
        "Temporal_Preop_Sens_cal": tv_pre["sens_cal"],
        "Temporal_Preop_Spec_cal": tv_pre["spec_cal"],
        "Temporal_Preop_F1_cal": tv_pre["f1_cal"],
        "Temporal_Preop_Brier_cal": tv_pre["brier_cal"],
        "Temporal_Preop_CalSlope": tv_pre["cal_slope"],
        "Temporal_Preop_CalIntercept": tv_pre["cal_intercept"],
    })

    # Final models for SHAP (fit on full cohort; for interpretability figures)
    m_full = lgb.LGBMClassifier(**LGB_PARAMS); m_full.fit(X_full, y)
    m_pre  = lgb.LGBMClassifier(**LGB_PARAMS); m_pre.fit(X_preop, y)
    models_full[tp] = m_full
    models_pre[tp]  = m_pre
    X_full_tp[tp]   = X_full
    X_pre_tp[tp]    = X_preop

    plot_shap_top7_boxed(m_full, X_full,  OUT_DIR / f"SHAP_Top7_{tp}_Full_x{SHAP_XMAX}.png",  x_max=SHAP_XMAX)
    plot_shap_top7_boxed(m_pre,  X_preop, OUT_DIR / f"SHAP_Top7_{tp}_PreopOnly_x{SHAP_XMAX}.png", x_max=SHAP_XMAX)

# SHAP heatmaps (no title)
shap_heatmap_3cols(models_full, X_full_tp, OUT_DIR / "SHAP_Heatmap_Full_3cols.png")
shap_heatmap_3cols(models_pre,  X_pre_tp,  OUT_DIR / "SHAP_Heatmap_Preop_3cols.png")

# Excel output
df_cv  = pd.DataFrame(rows_cv)
df_tv  = pd.DataFrame(rows_tv)
df_del = pd.DataFrame(rows_del)

df_delta = df_cv.merge(
    df_tv[["Timepoint","Temporal_Full_AUC_cal","Temporal_Preop_AUC_cal"]],
    on="Timepoint", how="left"
)
df_delta["DeltaAUC_TemporalMinusCV_Full"]  = df_delta["Temporal_Full_AUC_cal"]  - df_delta["Full_AUC"]
df_delta["DeltaAUC_TemporalMinusCV_Preop"] = df_delta["Temporal_Preop_AUC_cal"] - df_delta["Preop_AUC"]

xlsx_path = OUT_DIR / "Performance_Summary_COMPLETE_PAPER_VERSION_ESJ_UPDATED.xlsx"
with pd.ExcelWriter(xlsx_path) as w:
    df_cv.to_excel(w, sheet_name="CV_OOF_Isotonic", index=False)
    df_tv.to_excel(w, sheet_name="Temporal_Validation", index=False)
    df_delta.to_excel(w, sheet_name="CV_vs_Temporal_Delta", index=False)
    df_del.to_excel(w, sheet_name="DeLong_pvalues", index=False)

print("✅ Submission FINAL complete (Updated).")
print("📁 Outputs:", OUT_DIR)
print(" - ROC_* (Full/Preop)")
print(" - DCA_* (Full/Preop)  figsize=(8,6), y-ticks=0.1")
print(" - Calibration_* (Full/Preop) figsize=(6,6) label fs=20")
print(" - SHAP Top7_* (Full/Preop) label/ticks fs=20")
print(" - SHAP Heatmaps (Full/Preop) no title; xtick/ytick fs=20; cbar fs=18")
print(" - Excel:", xlsx_path)
print(f"Primary Bonferroni alpha = {ALPHA_PRIMARY_BONF:.4f}")

ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/tmp/ipython-input-2770561304.py:413: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = dfc.groupby("bin").agg(obs=("y","mean"), pred=("p","mean"))
/tmp/ipython-input-2770561304.py:413: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = dfc.groupby("bin").agg(obs=("y","mean"), pred=("p","mean"))


ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/tmp/ipython-input-2770561304.py:440: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, X_disp, plot_type="bar", max_display=7, show=False, color="dimgray")
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/tmp/ipython-input-2770561304.py:440: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence t

ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

...

ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/tmp/ipython-input-2770561304.py:413: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = dfc.groupby("bin").agg(obs=("y","mean"), pred=("p","mean"))
/tmp/ipython-input-2770561304.py:413: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = dfc.groupby("bin").agg(obs=("y","mean"), pred=("p","mean"))


ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/tmp/ipython-input-2770561304.py:440: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, X_disp, plot_type="bar", max_display=7, show=False, color="dimgray")
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/tmp/ipython-input-2770561304.py:440: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence t

✅ Submission FINAL complete (Updated).
📁 Outputs: /content/ESJ_outputs
 - ROC_* (Full/Preop)
 - DCA_* (Full/Preop)  figsize=(8,6), y-ticks=0.1
 - Calibration_* (Full/Preop) figsize=(6,6) label fs=20
 - SHAP Top7_* (Full/Preop) label/ticks fs=20
 - SHAP Heatmaps (Full/Preop) no title; xtick/ytick fs=20; cbar fs=18
 - Excel: /content/ESJ_outputs/Performance_Summary_COMPLETE_PAPER_VERSION_ESJ_UPDATED.xlsx
Primary Bonferroni alpha = 0.0083
